# Async (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#!import ../nbs/Common.fs

In [ ]:
open Common

## choice

In [ ]:
let inline choice asyncs = async {
    let e = Event<_> ()
    use cts = new System.Threading.CancellationTokenSource ()
    let fn =
        asyncs
        |> Seq.map (fun a -> async {
            let! x = a
            e.Trigger x
        })
        |> Async.Parallel
        |> Async.Ignore
    Async.Start (fn, cts.Token)
    let! result = Async.AwaitEvent e.Publish
    cts.Cancel ()
    return result
}

## map

In [ ]:
let inline map fn a = async {
    let! x = a
    return fn x
}

## catch

In [ ]:
let inline catch a =
    a
    |> Async.Catch
    |> map (function
        | Choice1Of2 result -> Ok result
        | Choice2Of2 ex -> Error ex
    )

## runWithTimeoutAsync

In [ ]:
let inline runWithTimeoutAsync (timeout : int) fn =
    let getLocals () = $"timeout: {timeout} / {getLocals ()}"

    let timeoutTask = async {
        do! Async.Sleep timeout
        trace Debug (fun () -> "runWithTimeoutAsync") getLocals
        return None
    }

    let task = async {
        try
            let! result = fn
            return Some result
        with
        | :? System.AggregateException as ex when
            ex.InnerExceptions
            |> Seq.exists (function :? System.Threading.Tasks.TaskCanceledException -> true | _ -> false)
            ->
            let getLocals () = $"ex: {ex |> printException} / {getLocals ()}"
            trace Warning (fun () -> "runWithTimeoutAsync") getLocals
            return None
        | ex ->
            trace Critical (fun () -> $"runWithTimeoutAsync** / ex: {ex |> printException}") getLocals
            return None
    }

    [ timeoutTask; task ]
    |> choice

In [ ]:
let inline runWithTimeout timeout fn =
    fn
    |> runWithTimeoutAsync timeout
    |> Async.RunSynchronously

In [ ]:
//// test

Async.Sleep 60
|> runWithTimeout 10
|> _equal None

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 10
<null>


In [ ]:
//// test

Async.Sleep 10
|> runWithTimeout 60
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>


In [ ]:
//// test

async {
    raise (exn "error")
}
|> runWithTimeout 60
|> _equal None

00:00:00 #2 [Critical] runWithTimeoutAsync** / ex: System.Exception: error / timeout: 60
<null>


## runWithTimeoutChildAsync

In [ ]:
let inline runWithTimeoutChildAsync (timeout : int) fn = async {
    let getLocals () = $"timeout: {timeout} / {getLocals ()}"
    let! child = Async.StartChild (fn, timeout)
    return!
        child
        |> catch
        |> map (function
            | Ok result -> Some result
            | Error (:? System.TimeoutException as ex) ->
                trace Debug (fun () -> $"runWithTimeoutChildAsync") getLocals
                None
            | Error ex ->
                trace Critical (fun () -> $"runWithTimeoutChildAsync** / ex: {ex |> printException}") getLocals
                None
        )
}

In [ ]:
let inline runWithTimeoutChild timeout fn =
    fn
    |> runWithTimeoutChildAsync timeout
    |> Async.RunSynchronously

In [ ]:
//// test

Async.Sleep 60
|> runWithTimeoutChild 10
|> _equal None

00:00:00 #3 [Debug] runWithTimeoutChildAsync / timeout: 10
<null>


In [ ]:
//// test

Async.Sleep 10
|> runWithTimeoutChild 60
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>


In [ ]:
//// test

async {
    raise (exn "error")
}
|> runWithTimeoutChild 60
|> _equal None

00:00:00 #4 [Critical] runWithTimeoutChildAsync** / ex: System.Exception: error / timeout: 60
<null>


## runWithTimeoutStrict

In [ ]:
let inline runWithTimeoutStrict (timeout : int) fn =
    let getLocals () = $"timeout: {timeout} / {getLocals ()}"

    let timeoutTask = async {
        do! Async.Sleep timeout
        return None, getLocals
    }
    
    let task = async {
        try
            return Async.RunSynchronously (fn, timeout) |> Some, getLocals
        with
        | :? System.TimeoutException as ex ->
            let getLocals () = $"ex: {ex |> printException} / {getLocals ()}"
            return None, getLocals
        | ex ->
            trace Critical (fun () -> $"runWithTimeoutStrict / ex: {ex |> printException}") getLocals
            return raise ex
    }

    try
        [| timeoutTask; task |]
        |> Array.map Async.StartAsTask
        |> System.Threading.Tasks.Task.WhenAny
        |> fun task ->
            match task.Result.Result with
            | None, getLocals ->
                trace Debug (fun () -> "runWithTimeoutStrict") getLocals
                None
            | result, _ -> result
    with
    | :? System.AggregateException as ex when
        ex.InnerExceptions
        |> Seq.exists (function :? System.Threading.Tasks.TaskCanceledException -> true | _ -> false)
        ->
        let getLocals () = $"ex: {ex |> printException} / {getLocals ()}"
        trace Warning (fun () -> "runWithTimeoutStrict") getLocals
        None
    | ex ->
        let getLocals () = $"ex: {ex |> printException} / {getLocals ()}"
        trace Critical (fun () -> "runWithTimeoutStrict**") getLocals
        None

In [ ]:
//// test

Async.Sleep 60
|> runWithTimeoutStrict 10
|> _equal None

00:00:00 #5 [Debug] runWithTimeoutStrict / ex: System.TimeoutException: The operation has timed out. / timeout: 10
<null>


In [ ]:
//// test

Async.Sleep 10
|> runWithTimeoutStrict 60
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>


In [ ]:
//// test

async {
    raise (exn "error")
}
|> runWithTimeoutStrict 60
|> _equal None

00:00:01 #6 [Critical] runWithTimeoutStrict / ex: System.Exception: error / timeout: 60
00:00:01 #7 [Critical] runWithTimeoutStrict** / ex: System.AggregateException: One or more errors occurred. (error) / timeout: 60
<null>


## awaitValueTask

In [ ]:
let inline awaitValueTaskUnit (task : System.Threading.Tasks.ValueTask) =
    task.AsTask () |> Async.AwaitTask

let inline awaitValueTask (task : System.Threading.Tasks.ValueTask<_>) =
    task.AsTask () |> Async.AwaitTask

## init

In [ ]:
let inline init x = async {
    return x
}

In [ ]:
//// test

init 1
|> Async.RunSynchronously
|> _equal 1

1


## mergeCancellationTokenWithDefaultAsync

In [ ]:
let inline mergeCancellationTokenWithDefaultAsync (token : System.Threading.CancellationToken) = async {
    let! ct = Async.CancellationToken
    let dct = Async.DefaultCancellationToken
    let cts = System.Threading.CancellationTokenSource.CreateLinkedTokenSource [| ct; dct; token |]
    return cts.Token
}

## withCancellationToken

In [ ]:
let inline withCancellationToken (ct : System.Threading.CancellationToken) fn =
    Async.StartImmediateAsTask (fn, ct)
    |> Async.AwaitTask

In [ ]:
//// test

let cts = new System.Threading.CancellationTokenSource ()

async {
    let run = async {
        do! Async.Sleep 100
        return 1
    }

    let! child =
        run
        |> withCancellationToken cts.Token
        |> catch
        |> Async.StartChild

    do! Async.Sleep 50
    cts.Cancel ()
    return! child
}
|> Async.RunSynchronously
|> Result.mapError (fun x -> x.Message)
|> _equal (Error ("A task was canceled."))

FSharpResult<Int32,String>
      ResultValue: 0
      ErrorValue: A task was canceled.


## withTraceLevel

In [ ]:
let inline withTraceLevel level fn = async {
    let oldTraceLevel = traceLevel
    try
        traceLevel <- level
        return! fn
    finally
        traceLevel <- oldTraceLevel
}